In [65]:
import numpy as np

def writePitches(fn,inds,tempo=82,instrument=[0,0],add21=True,start_at= [0,0],durationsInQuarterNotes=False):
    from MidiFile import MIDIFile

    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 1   # In beats # In BPM
    volume   = 116 # 0-127, as per the MIDI standard

    ni = len(inds)
    MyMIDI = MIDIFile(ni,adjust_origin=False) # One track, defaults to format 1 (tempo track
                     # automatically created)
    MyMIDI.addTempo(track,time, tempo)


    for k in range(ni):
        MyMIDI.addProgramChange(k,k,0,instrument[k])


    times = start_at
    for k in range(len(inds)):
        channel = k
        track = k
        for i in range(len(inds[k])):
            pitch,duration,volume,isPause,tempo = inds[k][i]
            #print(pitch,duration,volume,isPause)
            track = k
            channel = k
            if not durationsInQuarterNotes:
                duration = 4*duration#*maxDurations[k] #findNearestDuration(duration*12*4)            
            #print(k,pitch,times[k],duration,100)
            if not isPause: #rest
                #print(volumes[i])
                # because of median:
                pitch = int(np.floor(pitch))
                if add21:
                    pitch += 21
                #print(pitch,times[k],duration,volume,isPause)    
                MyMIDI.addTempo(track,times[k], tempo)
                MyMIDI.addNote(track, channel, int(pitch), float(times[k]) , float(duration), int(volume))
                times[k] += duration*1.0  
            else:
                times[k] += duration*1.0
       
    with open(fn, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    print("written")  


cmajor = [60,63,67]    
dminor = [62,65,69]
fmajor = [65,69,72]
gmajor = [67,71,74]
bb = [dminor,fmajor,gmajor,cmajor]

G = [Permutation(x) for x in list(SymmetricGroup(3))]

S6 = [Permutation(x) for x in list(SymmetricGroup(6))]

ll1 = []
ll2 = []
c6 = 0
octave1 = 0
octave2 = -2
dur1 = 1/3
dur2 = 1/9
rndbits = [int(x)%2 for x in (str(pi.n(100000)))[2:]]
c = 0
lastbit = 0
tempo = 3**3
for p6 in S6:
    c3 = 0
    for p3 in p6:
        x3 = 0
        for x in G[p3-1]:
            ll1.append((bb[(c//45)%4][(x+1)%3]+octave1*12,dur1*1/3,64-12*c3,False,tempo))
            ll2.append((bb[(c//45)%4][(x-1)%3]+octave2*12,dur2*1/3,64+12*x3,False,tempo))
            x3 += 1
        c3+=1
    if octave1 == 0:
        sign1 = 1
    if octave1 == 2:
        sign1 = -1
    bit = rndbits[c]
    if bit==0:
        octave1 = min(max(octave1+sign1,0),2)
    factor1 = 1
    factor2 = 1
    if (lastbit,bit) == (1,1):
        factor1 = 2
        factor2 = 1/2
    if (lastbit,bit) == (0,0):
        factor1 = 1/2
        factor2 = 2
    #print(factor1,factor2,lastbit,bit,dur1,dur2)    
    #dur1 = min(max(factor1*dur1,1/2**3),1)
    #dur2 = min(max(factor2*dur2,1/2**2),2)
    if bit==1:
        tmp = dur2
        dur2 = dur1
        dur1 = tmp            
    if octave2 == -2:
        sign2 = 1
    if octave2 == 2:
        sign2 = -1        
    
    if lastbit==1:
        octave2 = min(max(octave2+sign2,-2),2)
    c+=1
    lastbit = bit
#print(ll)            

iinds = [ll1,ll2]    
          


fn = "./midi/experiment_with_permutations.mid"    
writePitches(fn,iinds,tempo=tempo,instrument=len(iinds)*[0],add21=False,start_at= len(iinds)*[0],durationsInQuarterNotes=False)    

written
